In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [112]:
# Open the data file
File_Path = '/Users/nv83/Dropbox/Research/Vacanti_Laboratory/projects/Alzheimers/MouseAstrocytes_oAB_INSRinhibition/inEFP/Astrocytes_inEFP.AIA/RelativeFluxes.xlsx'
RawData = pd.read_excel(File_Path, sheet_name='MIDs')

# Set the row names
RawData.index = RawData.loc[:,'Unnamed: 0']
RawData.drop('Unnamed: 0', axis=1, inplace=True)

# Set the column names and drop unwanted columns
RawData.columns = RawData.iloc[1,:]
ColumsToDrop = ['fm3-2','theoretical','nothing']
[RawData.drop(column,axis=1, inplace=True) for column in ColumsToDrop]


# Filter out unneeded rows
# Drop rows with nothing in them
RawData = RawData[RawData.index.notnull()]

# Get the row location of where the MIDs and Peak Areas start
MID_index = RawData.index.get_loc('MIDs')
AreaIndex = RawData.index.get_loc('Peak Areas (ion counts * seconds)')

# Inidcate which fragments need the peak areas to compute ratios
SpecialFragments = ['lactate_233', 'lactate_261', 'HPA_261']

# Perform the row filtering
RowsToKeep = [((i>MID_index) | (RawData.index[i] in SpecialFragments)) & (i>AreaIndex) for i in range(len(RawData.index))]
RawData = RawData[RowsToKeep]

In [126]:
# Create a data frame to contain the ratios of metabolites to internal standards
Ratios = pd.DataFrame(index=RawData.index, columns=RawData.columns)

# Remove rows corresponding to the peak areas for fragments that rely on peak areas
[Ratios.drop(fragment, axis=0, inplace=True) for fragment in SpecialFragments]

# Rename the rows to correspond to fragments (i.e. remove the M0 or M1)
Ratios.index = [i.split(' ')[0] for i in Ratios.index]

# Drop duplicate row names - each fragment had a name that came from the M0 and M1 forms of the fragment
Ratios = Ratios[~Ratios.index.duplicated(keep='first')]

# After removing the M0 and M1, the row corresponding to the internal standard for lactate needs to be removed
Ratios.drop('HPA_261', axis=0, inplace=True)

# Compute the ratios for lactate independently because it relies on peak area ratios
Ratios.loc['lactate_233',:] = RawData.loc['lactate_233',:]/RawData.loc['HPA_261',:]
Ratios.loc['lactate_261',:] = RawData.loc['lactate_261',:]/RawData.loc['HPA_261',:]

# Compute the ratios for the remaining fragments
for i in Ratios.index:
    if i not in SpecialFragments:
        numerator = RawData.loc[i+' M0',:]
        denominator = RawData.loc[i+ ' M1',:]
        Ratios.loc[i,:] = np.divide(numerator.to_numpy(), denominator.to_numpy(), out=np.full_like(numerator.to_numpy(), np.nan), where=denominator.to_numpy()!=0)